In [1]:
%pwd

'd:\\Documentation_sample\\research'

In [2]:
import os 

os.chdir("../")

In [3]:
%pwd

'd:\\Documentation_sample'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TestingConfig:
    root_dir: Path
    trained_model_path : Path
    test_loader_dir :  Path
    params_random_seed: int

In [5]:
from sample_project.constants import *
from sample_project.utils.common import read_yaml, create_directories
import torch

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_testing_config(self) -> TestingConfig:
        testing = self.config.testing
        params = self.params
        create_directories([
            Path(testing.root_dir)
        ])

        testing_config = TestingConfig(
            root_dir=Path(testing.root_dir),
            trained_model_path=Path(testing.trained_model_path),
            test_loader_dir=Path(testing.test_loader_dir),
            params_random_seed = params.RANDOM_SEED
        )

        return testing_config


In [7]:
import urllib.request as request
from sample_project import logger
import numpy as np
import torch
from sample_project.utils.common import accuracy
from torchvision import datasets
import matplotlib.pyplot as plt
import torch.utils.data as data
import torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support


In [17]:
class Testing:
    def __init__(self, config: TestingConfig):
        self.config = config
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\
    

    def loading_iterators(self):
        self.test_iterator = torch.load(self.config.test_loader_dir)

    def initializing_model(self):
        self.model = torch.load(self.config.trained_model_path)
        self.model = self.model.to(self.device)  # Move model to the specified device
        self.loss_fn = nn.CrossEntropyLoss()

    def test(self):
        self.test_loop(self.model,self.test_iterator,self.loss_fn)



    @staticmethod    
    def test_loop(model,test_iterator,Loss_function):
        device = next(model.parameters()).device 
        model.eval()
        loss_vals = []
        acc_vals = []
        prec_vals = []
        recall_vals = []
        f1_vals= []
        with torch.no_grad():
            for iteration, idata in enumerate(test_iterator,1):
                
                inputs,targets = idata
                inputs,targets = inputs.to(device),targets.to(device)
                
                outputs = model(inputs)
                
                current_loss = Loss_function(outputs,targets)
                current_accuracy = accuracy(outputs,targets)
                
                outputs_np = outputs.cpu().detach().numpy()
                targets_np = targets.cpu().detach().numpy()
                
                precision, recall, f1, _ = precision_recall_fscore_support(
                    targets_np, np.argmax(outputs_np, axis=1), average='weighted')
                
                loss_vals.append(float(current_loss))
                acc_vals.append(float(current_accuracy))
                prec_vals.append(precision)
                recall_vals.append(recall)
                f1_vals.append(f1)

                print("\rEvaluating the model: {}/{} ({:.1f}%) Loss: {:.5f} accuracy: {:.5f}".format(
                        iteration, len(test_iterator),
                        iteration * 100 / len(test_iterator),current_loss,current_accuracy),
                    end=" " * 10)
                
        avg_loss = np.mean(loss_vals)
        avg_acc = np.mean(acc_vals)
        avg_precision = np.mean(prec_vals)
        avg_recall = np.mean(recall_vals)
        avg_f1 = np.mean(f1_vals)
        
        logger.info("\rTest Results: Loss: {:.6f}, Accuracy: {:.4f}%, Precision: {:.4f}, Recall: {:.4f}, F1: {:.4f}".format(
            avg_loss, avg_acc * 100, avg_precision, avg_recall, avg_f1))


In [18]:
from sklearn.exceptions import UndefinedMetricWarning
import warnings

warnings.filterwarnings('ignore', category=UndefinedMetricWarning) 


In [19]:
try:
    config = ConfigurationManager()
    Testing_config = config.get_testing_config()
    Testing_step = Testing(config=Testing_config)
    Testing_step.loading_iterators()
    Testing_step.initializing_model()
    Testing_step.test()
    
except Exception as e:
    raise e

[2024-07-29 18:01:33,350: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-29 18:01:33,351: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-29 18:01:33,352: INFO: common: created directory at: artifacts]
[2024-07-29 18:01:33,354: INFO: common: created directory at: artifacts\testing]
Evaluating the model: 43/157 (27.4%) Loss: 2.30287 accuracy: 0.15625          

KeyboardInterrupt: 